In [1]:
import pandas as pd
import os

In [24]:
# loading data sets
countries_names_to_code = pd.read_csv(os.path.join('data',r'countryarea.csv'))[['Country Name','Country Code']]

votes = pd.read_csv(os.path.join('data',r'votes_with_minorities.csv'))
votes.drop(columns=['Unnamed: 0'],inplace=True)
print(countries_names_to_code.head())
print(len(countries_names_to_code))
print(votes.head())
print(len(votes))

  Country Name Country Code
0        Aruba          ABW
1  Afghanistan          AFG
2       Angola          AGO
3      Albania          ALB
4      Andorra          AND
264
   Year (semi-) final Jury or Televoting From country To country  Points  \
0  1975             f                  J      Belgium    Belgium       0   
1  1976             f                  J      Belgium    Belgium       0   
2  1977             f                  J      Belgium    Belgium       0   
3  1978             f                  J      Belgium    Belgium       0   
4  1979             f                  J      Belgium    Belgium       0   

   2_minority_in_1  
0              0.0  
1              0.0  
2              0.0  
3              0.0  
4              0.0  
38181


In [35]:
# adding code to FROM countries
votes_with_code = pd.merge(votes, countries_names_to_code, how='inner', left_on=['From country'], right_on=['Country Name'])
votes_with_code.drop(columns=['Country Name'],inplace=True)
votes_with_code.rename(columns={"Country Code": "From country code"},inplace=True)

# adding code to TO countries
votes_with_code = pd.merge(votes_with_code, countries_names_to_code, how='inner', left_on=['To country'], right_on=['Country Name'])
votes_with_code.drop(columns=['Country Name'],inplace=True)
votes_with_code.rename(columns={"Country Code": "To country code"},inplace=True)
votes_with_code = votes_with_code[['Year', '(semi-) final', 'Jury or Televoting','From country', 'From country code', 'To country', 
                   'To country code','Points', '2_minority_in_1']]


,Year,(semi-) final,Jury or Televoting,From country,From country code,To country,To country code,Points,2_minority_in_1
50,1986,f,J,Finland,FIN,Belgium,BEL,12,0.0
51,1987,f,J,Finland,FIN,Belgium,BEL,3,0.0
52,1988,f,J,Finland,FIN,Belgium,BEL,0,0.0
53,1989,f,J,Finland,FIN,Belgium,BEL,0,0.0
54,1990,f,J,Finland,FIN,Belgium,BEL,0,0.0
55,1991,f,J,Finland,FIN,Belgium,BEL,3,0.0
56,1992,f,J,Finland,FIN,Belgium,BEL,0,0.0
57,1993,f,J,Finland,FIN,Belgium,BEL,0,0.0
58,1996,f,J,Finland,FIN,Belgium,BEL,0,0.0
59,1998,f,J,Finland,FIN,Belgium,BEL,0,0.0


In [37]:
# loading distances 
countries_distw = pd.read_csv(os.path.join('data',r'countries_distw.csv'))[['iso_o','iso_d','distw']]


FileNotFoundError: [Errno 2] File b'data\\countries_distw.csv' does not exist: b'data\\countries_distw.csv'